# CHATBOTS - Using Natural Language Processing and Tensorflow

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import random


import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [14]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the data to convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......
['?']


In [4]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
44 documents
17 classes ['AIchatbot', 'Common1', 'Common2', 'Common3', 'DOM', 'Facemask_Detector', 'Frontend', 'Image-to-TextConverter ', 'Models', 'Personalinformation', 'React', 'about', 'goodbye', 'greeting', 'machinelearning', 'project', 'thanks']
74 unique stemmed words ["'s", '.', 'about', 'ai', 'and', 'anyon', 'apply', 'ar', 'bas', 'both', 'brief', 'bye', 'chatbot', 'convert', 'cre', 'day', 'deploy', 'describ', 'detect', 'diff', 'docu', 'doe', 'dom', 'explain', 'facemask', 'feat', 'for', 'framework', 'frontend', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'how', 'ide', 'image-to', 'image-to-text', 'in', 'is', 'kes', 'kind', 'langu', 'lat', 'learn', 'machin', 'mak', 'me', 'model', 'object', 'of', 'opencv', 'platform', 'project', 'python', 'react', 'reacts', 'see', 'tel', 'tensorflow', 'text', 'thank', 'that', 'the', 'ther', 'thi', 'two', 'us', 'websit', 'what', 'who', 'yo', 'you']


In [5]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [6]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
tf.compat.v1.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [7]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [8]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [9]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')


Training Step: 5999  | total loss: 0.01110 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.01110 - acc: 1.0000 -- iter: 40/44
Training Step: 6000  | total loss: 0.01072 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.01072 - acc: 1.0000 -- iter: 44/44
--
Saving the Model.......
INFO:tensorflow:C:\Users\Xyron\Documents\Machine Learning\ChatBot_Tensorflow_NLP\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [11]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from C:\Users\Xyron\Documents\Machine Learning\ChatBot_Tensorflow_NLP\model.tflearn


In [12]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [13]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- Hi
Hello, thanks for visiting
You- Hi
Hi there, how can I help?
You- Byee
See you later, thanks for visiting
You- Thanks
My pleasure
You- Tell me about this website
It is a website based on different machine learning models
You- What Models are used in this website
It is a website based on different machine learning models
You- what models are used
We used CNN for mask detection, SEQ2SEQ model for chatbot and Image Processing Model in out Text to image Converter
You- What is machine learning
Machine learning is the study of computer algorithms that can improve automatically through experience and by the use of data. It is seen as a part of artificial intelligence.
You- What does you project do
our project covers three main ML and ReactJS based applications -The Facemask Detector -AI ChatBot -Image-to-text Converter
You- Tell me about facemask detector
React. js/React is an open-source front-end framework that is based on JavaScript, developed by Facebook, and best known for its vi

KeyboardInterrupt: Interrupted by user